In [ ]:
!pip install pymystem3
!pip install nltk

In [ ]:
from string import punctuation
import re
import csv
import sys
from pymystem3 import Mystem
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Открываю файл text.csv и разделяю на предложения, сохраняю в sentences

In [ ]:
csv.field_size_limit(sys.maxsize)
sentences = []
with open("text.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        clear_txt = re.sub("\*.*(\(Примеч. Лермонтова\)|Ред). ", "", row[0])  # убираем Примечания, потому что они посреди других предложений находятся
        sentences.extend([(i.strip(), row[1]) for i in nltk.sent_tokenize(clear_txt)])

Определяю морфологию с помощью mystem.
Сохраняю данный в следующем формате для поиска:

Мама мыла раму., (мама; мыла; раму), (мама; мыть; рама), (Noun; Verb; Noun), Про мытьё рамы. Глава 1

(Скобки и пробелы только чтобы было лучше понятно здесь, их не будет в файле)

In [ ]:
sentences.pop(0) # там строка заголовка

('Text', 'Part')

In [ ]:
punctuation += '«»—'

In [ ]:
mystem = Mystem()
rows = []
for sent, chapter in sentences:
    words = []
    lemmas = []
    pos = []
    words_analyze = mystem.analyze(sent)
    for i in words_analyze:
        if i['text'].strip() != " " and i['text'].strip() not in punctuation:
            words.append(i['text'].lower())
            if 'analysis' in i and len(i['analysis']) != 0:
                lemmas.append(i['analysis'][0].get('lex', ""))
                pos.append(i['analysis'][0].get('gr', '').split('=')[0].split(",")[0])  # два split, потому что после части речи м. б. = или ,
            else:
                lemmas.append(i['text'].lower())
                pos.append('')
    words_line = ";".join(words)
    lemmas_line = ';'.join(lemmas)
    pos_line = ';'.join(pos)
    rows.append([sent, words_line, lemmas_line, pos_line, chapter])


In [ ]:
with open("corpus_data.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Sentence', 'Tokens', 'Lemmas', 'POS', 'Chapter'])
    writer.writerows(rows)